Copyright by Paul Rudolph

Research Group Applied Systems Biology - Head: Prof. Dr. Marc Thilo Figge

https://www.leibniz-hki.de/en/applied-systems-biology.html

HKI-Center for Systems Biology of Infection

Leibniz Institute for Natural Product Research and Infection Biology - Hans Knöll Insitute (HKI)

Adolf-Reichwein-Straße 23, 07745 Jena, Germany

This code is licensed under BSD 2-Clause

See the LICENSE file provided with this code for the full license.

In [ ]:
import os
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib
import pandas as pd
import numpy as np
import multiprocessing,functools
from tqdm import tqdm
import sys
from scipy.stats import chi2
from SALib.sample import saltelli
from SALib.analyze import sobol

from model import system_size,adjust_CaL,time_scaling,ldh_model,cal_model,cal_nb_model,candida_model,cal_nb_with_binding_model
from fitting import simulate, validation_interval,distance,distance_max,log_distance, get_parameters
from model_utils import objective_LDH,objective_CAL,objective_Nb,generate_data_LDH,generate_data_CAL,generate_data_Nb,prepare_data_LDH,prepare_data_CaL,prepare_data_Nb,objective_Candida,prepare_data_Candida
from preprocessing import read_data, preprocess_data

%load_ext autoreload
%autoreload 2

RESULT_FOLDER = os.path.abspath(f"../PUBLICATION/SIMULATION/")
synthetic_nb = "CAL1-H1"
native_nb = "CAL1-F1"
PLOT_FIT_FOLDER = "../PUBLICATION/PLOTS/"
os.makedirs(PLOT_FIT_FOLDER,exist_ok=True)
DATA_FOLDER = os.path.abspath("../DATA/")
file_name = os.path.join(DATA_FOLDER,"Experimental Data For Modelling.xlsx")
df_ex5,df_ex3,df_ex3_candida,df_ex1,df_fig_cal,df_fig_candida = read_data(file_name)
df_data_LDH,df_data_sim,df_data_pre,df_data_pre_co,df_data_post,df_data_candida_sim,df_data_candida_pre,df_data_candida_post = preprocess_data(file_name,time_scaling, adjust_CaL=adjust_CaL, adjust_LDH=True,system_size=system_size)

## Set style for plotly plots
  * Font: 12px(16pt), Arial
  * Ticklabels: 11px(14.66pt),bold,black
  * Horizontal legend at the bottom of the figure
  * Bold axis lines and bold ticks
  * Custom function for line plots with error bands

In [ ]:
sns.set_style("whitegrid")
plotly_template = pio.templates["simple_white"]
plotly_template.layout.update(font={"size":14+2/3, "family":"Arial"},
                              legend_title=dict(font=dict(size=16)))
plotly_template.layout.legend.update(font=dict(size=16),
                                     orientation='h',
                                     xanchor='center',
                                     yanchor='bottom',
                                     x=0.5,
                                     y=-0.15,
                                     itemwidth=40,
                                     itemsizing="constant",
                                     entrywidth=40)
plotly_template.layout.update(xaxis=dict(linecolor="Black",tickprefix="<b>",ticksuffix="</b>",tickangle=0, linewidth=2,tickcolor="black",tickwidth=2),
                              font=dict(size=14.66),
                              titlefont=dict(size=16,family="Arial"))
plotly_template.layout.update(yaxis=dict(linecolor="Black",tickprefix="<b>",ticksuffix="</b>",tickangle=0, linewidth=2,tickcolor="black",tickwidth=2),
                              font=dict(size=14.66),
                              titlefont=dict(size=16,family="Arial"))
pio.templates["plotly_custom"] = plotly_template
px.defaults.template = "plotly_custom"
px.defaults.width = 600
px.defaults.height = 400

In [ ]:
def line_with_bands(**kwargs):
    '''Plotly line plot with error bands'''
    figure_with_errok_bars = px.line(**kwargs)
    fig = px.line(**{arg: val for arg,val in kwargs.items() if arg != 'error_y'})
    for data in figure_with_errok_bars.data:
        x = list(data['x'])
        y_upper = list(data['y'] + data['error_y']['array'])
        y_lower = list(np.maximum(data['y'] - data['error_y']['array'],0.0) if data['error_y']['arrayminus'] is None else data['y'] - data['error_y']['arrayminus'])
        color = f"rgb{matplotlib.colors.to_rgb(data['line']['color'])}".replace("rgb","rgba").replace(")",",.3)")
        fig.add_trace(
            go.Scatter(
                x = x+x[::-1],
                y = y_upper+y_lower[::-1],
                fill = 'toself',
                fillcolor = color,
                line = dict(
                    color = 'rgba(255,255,255,0)'
                ),
                hoverinfo = "skip",
                showlegend = False,
                legendgroup = data['legendgroup'],
                xaxis = data['xaxis'],
                yaxis = data['yaxis'],
            )
        )
    reordered_data = []
    for i in range(int(len(fig.data)/2)):
        reordered_data.append(fig.data[i+int(len(fig.data)/2)])
        reordered_data.append(fig.data[i])
    fig.data = tuple(reordered_data)
    return fig

In [ ]:
def generate_df(RESULT_FOLDER):
    '''Load in the fitting results (always the best evaluation)'''
    df_list = list()
    for result in os.listdir(RESULT_FOLDER):
            result_path = os.path.join(RESULT_FOLDER, result)
            if os.path.isdir(result_path):
                df = {**pd.read_feather(result_path+"/LDH.feather").assign(A= int(result)).iloc[0].rename({"Fun":"LDH_ERROR"}),
                                **pd.read_feather(result_path+"/CAL.feather").iloc[0].rename({"Fun":"CAL_ERROR"}),
                                **pd.read_feather(result_path+f"/NB_CAL.feather").iloc[0].rename({"Fun":"NB_ERROR"}),
                     **pd.read_feather(result_path+f"/CANDIDA.feather").iloc[0].rename({"Fun":"CANDIDA_ERROR"})}
                df_list.append(df)
    return pd.DataFrame(df_list).sort_values("A").assign(k_c=0.0)

def simulate_data_for_dosing(model,args):
    '''Simulate model for a specific CaL and Nb concentration'''
    initials,kwargs,CaL,Nb = args
    column_names = ["CaL","Nb","CaL8","E_Alive","E_Dead","LDH"]
    time_max = 100*time_scaling
    time_points = np.linspace(0,time_max,time_max+1,dtype="float64")
    funcptr = model
    order_parameters = ['k_a', 'k_c', 'k_n','k_l','k_b', 'k_d','A', 'alpha', 'beta']
    parameters = np.array([kwargs[op] for op in order_parameters])
    return pd.DataFrame(simulate(parameters, initials, time_points,funcptr).T,columns = column_names).assign(Time = time_points, CaL = CaL, Nb = Nb, Agg =kwargs["A"])

def simulate_dosing(model, df):
    '''Screen over nb and cal concentration to find perfect dosing'''
    screening_list = list()
    for row in df.to_dict(orient='records'):
        for CaL in np.linspace(0,120,121):
            for Nb in np.linspace(0,70,701):
                time_max = 100*time_scaling
                time_points = np.linspace(0,time_max,time_max+1,dtype="float64")
                initials = np.array([CaL/row["A"]*system_size*1e-3,Nb*system_size*1e-3,0.0,1.0,0.0,0.0])
                screening_list.append((initials,row,CaL, Nb))



    results = list()
    constant_args = [model]
    partial_parallelize = functools.partial(simulate_data_for_dosing, *constant_args)
    with multiprocessing.Pool(np.minimum(len(screening_list),250)) as pool:
        results = list(tqdm(pool.imap(partial_parallelize, screening_list), total=len(screening_list), file=sys.stdout))

    df_effect = (pd.concat(results)[["E_Dead","Time","CaL", "Nb","Agg"]])

    return df_effect

def generate_dosing_figure(df):
    '''Helper function for generating the final plot'''
    data = df.groupby(["CaL"])["Nb_max"].agg(["mean","std"]).reset_index().assign(std= lambda df_:df_["std"].where(df_["std"]!=0, 0.01)).sort_values("CaL",ascending=False)
    fig = line_with_bands(data_frame=data,
               x="CaL",y="mean", 
               labels={"Effect":" Dead VEC[%]","Time":"Time [h]","mean":f"<b>{synthetic_nb} (µM)</b>","CaL":"<b>CaL (µM)</b>","A":"Aggregate Size"}, 
               error_y="std")
    fig.update_yaxes(range=[0,61])
    fig.update_xaxes(range=[0,111],tickmode="array",tickvals=[0,10,20,30,40,50,60,70,80,90,100,110])
    return fig

# Fig 4a

In [ ]:
df_dosing = simulate_dosing(cal_nb_model.address, generate_df(RESULT_FOLDER))
df = (pd.concat([df_dosing])
     .query("E_Dead != 0 and Time == 24")
     .assign(E_Dead = lambda df_:df_.E_Dead.where(df_.E_Dead == 0,df_.eval("E_Dead*100")))
     .groupby(["CaL","Nb","Agg"])["E_Dead"].agg("max").reset_index()
      .query("E_Dead < 10 ")
     .groupby(["CaL","Agg"],group_keys=False).apply(lambda grp_: grp_.assign(Nb_max = grp_.query(f"Nb == {grp_.Nb.min()}").sort_values("Nb",ascending=True).Nb.iloc[0])) 
      [["Nb_max","CaL","Agg"]].drop_duplicates().reset_index(drop=True)
    )
fig = generate_dosing_figure(df)
fig.add_hline(y=50.63,opacity=1.,visible=True,line=dict(width=2,dash="longdash",color="rgb(0,0,0)"),x1=107/111)
fig.add_vline(x=107,opacity=1.,visible=True,line=dict(width=2,dash="longdash",color="rgb(0,0,0)"),y1=50.8/61)
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"FIG_4A.svg"), format="svg")
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"FIG_4A.png"), format="png")
fig.show()

# Fig 4b

In [ ]:
def simulate_data(funcptr,initials, time_points,**kwargs):
    '''Simulate model with a certain parmaater configuration'''
    order_parameters = ['k_a', 'k_c', 'k_n','k_l','k_b', 'k_d','A', 'alpha', 'beta',"k_s"]
    parameters = np.array([kwargs[op] for op in order_parameters])
    data = simulate(parameters, initials, time_points,funcptr)
    return data

df_list = list()
column_names = ["CaL","Nb","CaLA","E_Alive","E_Dead","LDH","Candida","F_NI","F_I"]
for funccptr,df in [(candida_model.address,generate_df(RESULT_FOLDER))]:
    for row in df.to_dict(orient='records'):
        for delay in np.arange(0,24):
            row.update(delay=delay)
            #row.update(k_b = row["k_b_c"])
            candida_coverage= df_data_candida_sim.query("Time==72").Readout.mean()/df.beta.mean() 
            time_delay = delay*time_scaling
            Y0 = 2.0e4
            initials = np.array([0.0,0,0.0,candida_coverage,0.0,0.0,Y0,0,0])
            if delay != 0.0:
                time_points = np.linspace(0,time_delay,time_delay+1,dtype="float64")
                post_initials = np.array([0.0,0,0.0,candida_coverage,0.0,0.0,Y0,0,0])
                initials = simulate_data(funccptr,post_initials, time_points, **row)[:,-1]
                df_list.append((pd.DataFrame(simulate_data(funccptr,post_initials, time_points, **row).T,columns = column_names)
                .assign(Delay = delay, 
                  Time = time_points,  
                  initNb = Nb, 
                  Agg= row["A"],
                  E_Alive= lambda df_:df_.E_Alive +(1-candida_coverage),
                  E_Dead=lambda df_:df_.E_Dead*100)))
            for Nb in [0,4,8,16]:
                setup_initials = np.copy(initials)
                time_max = 72*time_scaling
                time_points = np.linspace(0,time_max,time_max+1,dtype="float64")
                setup_initials[1] = Nb*system_size*1e-3
                df_list.append(pd.DataFrame(simulate_data(funccptr,setup_initials, time_points, **row).T,columns = column_names)
                          .assign(Delay = delay, 
                                  Time = time_points+delay,
                                  secretion= lambda df_: df_.eval(f"E_Alive*F_I*{row['k_s']}*{row['A']}"),
                                  initNb = Nb, 
                                  Agg= row["A"],
                                  E_Alive= lambda df_:df_.E_Alive +(1-candida_coverage),
                                  E_Dead=lambda df_:df_.E_Dead*100)
                 )
fig = line_with_bands(data_frame=(
    pd.concat(df_list).assign(secretion = lambda df_: df_.secretion*1e6)
    .query("Delay == 0 and Time <= 72 and initNb==0")
    .sort_values("Time")
    .groupby(["initNb","Delay","Agg"], group_keys=False).apply(lambda grp_: grp_.assign(secretion=lambda df_:df_.secretion.cumsum()))
    .groupby(["initNb","Time","Delay"])["secretion"].agg(["mean","std"]).reset_index()
), 
           x="Time",y="mean",  error_y="std", labels={"mean":'<b>"Effective" CaL (µM)<br>during infection</b>',"Time":"<b>Time (h)</b>","initNb":"Initial CAL1-F1 [µM]", "Delay":"Post Treatment [h]"})
fig.update_xaxes(
    range=[0,24.3],
    tickmode='array',
    tickvals=[0,8,16,24,32,40,48],  # Use all time points as tick values
)
fig.update_yaxes(range=[0,123])
fig.show()
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"FIG_4B.svg"), format="svg")
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"FIG_4B.png"), format="png")

# Fig 4c

In [ ]:
df_list = list()
column_names = ["CaL","Nb","CaLA","E_Alive","E_Dead","LDH","Candida","F_NI","F_I"]
for funcptr,df in [(candida_model.address,generate_df(RESULT_FOLDER))]:
    for row in df.to_dict(orient='records'):
        for delay in np.arange(0,24):
            row.update(delay=delay)
            candida_coverage= df_data_candida_sim.query("Time==72").Readout.mean()/df.beta.mean()
            time_delay = delay*time_scaling
            Y0 = 2.0e4
            initials = np.array([0.0,0,0.0,candida_coverage,0.0,0.0,Y0,0,0])

            for Nb in [0,4,8,16]:
                if delay != 0.0:
                    time_points = np.linspace(0,time_delay,time_delay*10+1,dtype="float64")
                    post_initials = np.array([0.0,0,0.0,candida_coverage,0.0,0.0,Y0,0,0])
                    initials = simulate_data(funcptr,post_initials, time_points, **row)[:,-1]
                    df_list.append((pd.DataFrame(simulate_data(funcptr,post_initials, time_points, **row).T,columns = column_names)
                    .assign(Delay = delay, 
                      Time = time_points,  
                      initNb = Nb, 
                      Agg= row["A"],
                      E_Alive= lambda df_:df_.E_Alive +(1-candida_coverage),
                      E_Dead=lambda df_:df_.E_Dead*100)))
                setup_initials = np.copy(initials)
                time_max = 72*time_scaling
                time_points = np.linspace(0,time_max,time_max*10+1,dtype="float64")
                setup_initials[1] = Nb*system_size*1e-3
                df_list.append(pd.DataFrame(simulate_data(funcptr,setup_initials, time_points, **row).T,columns = column_names)
                          .assign(Delay = delay, 
                                  Time = time_points+delay,  
                                  initNb = Nb, 
                                  Agg= row["A"],
                                  E_Alive= lambda df_:df_.E_Alive +(1-candida_coverage),
                                  E_Dead=lambda df_:df_.E_Dead*100)
                 )
times = np.arange(0,24)
delay_array= [0,3,9,12]

data = pd.concat(df_list).assign(LDH = lambda df_:df_.LDH *1e3).query("Time <= 48").groupby(["initNb","Time","Delay"])["LDH"].agg(["mean","std"]).reset_index()
time_10 = data.query('mean <= 10 and Delay ==0 and initNb==0').sort_values('Time').iloc[-1].Time             
fig = line_with_bands(data_frame=data.query(f"Delay in {delay_array}").assign(initNb= lambda df_:"<b>"+df_.initNb.map(str)+" µM</b>\t"), x="Time",y="mean",color="initNb", error_y="std",
                      width=1200,height=800, facet_col="Delay",
                      facet_col_wrap=2,facet_row_spacing=.2,facet_col_spacing=.12,
           labels={"mean":" LDH (ng/ml)","Time":"Time (h)","initNb":f"<b>{native_nb}:\t</b>","Delay":"post-treatment"})
fig.for_each_annotation(lambda anno: anno.update(text = " ".join(anno.text.__add__(" h").split("=")[::-1])))
fig.for_each_annotation(lambda anno: anno.update(text = "Simultaneous addition" if anno.text.split(" ")[0] == "0" else anno.text))

fig.add_hline(y=200,opacity=1.,visible=True,line=dict(width=3,dash="dash",color="rgba(255,0,0)"))
fig.update_xaxes(
    range=[0,24.3],showticklabels=True,
    tickmode='array',
    tickvals=np.arange(0,48,2)  # Use all time points as tick values
)

for yaxis in fig.layout:
    if "yaxis" in yaxis:
        fig.layout[yaxis].title.text = "<b>LDH (ng/ml)</b>"

for xaxis in fig.layout:
    if "xaxis" in xaxis:
        fig.layout[xaxis].title.text = "<b>Time (h)</b>"
fig.for_each_annotation(lambda anno: anno.update(text = "<b>"+anno.text+"</b>"))

fig.update_yaxes(range=[0,404],showticklabels=True)
fig.show()
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"FIG_4C.svg"), format="svg")
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"FIG_4C.png"), format="png")

# Fig S3

In [ ]:
df = generate_df(RESULT_FOLDER)
df_liste = list()
df_scatter = list()
column_names = ["CaL","Nb","CaLA","E_Alive","E_Dead","LDH"]
def simulate_data(initials, time_points,**kwargs):
    funcptr = cal_nb_model.address
    order_parameters = ['k_a', 'k_c', 'k_n','k_l',"k_b", 'k_d','A', 'alpha', 'beta']
    parameters = np.array([kwargs[op] for op in order_parameters])
    return simulate(parameters, initials, time_points,funcptr)
for row in df.to_dict(orient='records'):
    for CaL in [1,10,35,70]:
        for Nb in [0,4,8,16]:
            time_max = 72*time_scaling
            time_points = np.linspace(0,time_max,time_max+1,dtype="float64")
            initials = np.array([CaL/row["A"]*system_size*1e-3,Nb*system_size*1e-3,0.0,1.0,0.0,0.0])
            df_liste.append(pd.DataFrame(simulate_data(initials, time_points, **row).T,columns = column_names)
                      .assign(Time = time_points, CaL = CaL, Nb = Nb, Agg =row["A"])
                 )
column_names = ["CaL","Nb","CaLA","E_Alive","E_Dead","LDH","_1","_2","_3"]
def simulate_data(initials, time_points,**kwargs):
    funcptr = candida_model.address
    order_parameters = ['k_a', 'k_c', 'k_n','k_l',"k_b", 'k_d','A', 'alpha', 'beta',"k_s"]
    parameters = np.array([kwargs[op] for op in order_parameters])
    return simulate(parameters, initials, time_points,funcptr)
for row in df.to_dict(orient='records'): 
    for Nb in [0,4,8,16]:
        time_max = 72*time_scaling
        time_points = np.linspace(0,time_max,time_max+1,dtype="float64")
        initials = np.array([0,Nb*system_size*1e-3,0.0,df_data_candida_sim.query("Time==72").Readout.mean()/df.beta.mean(),0.0,0.0,20000,0,0])
        df_liste.append(pd.DataFrame(simulate_data(initials, time_points, **row).T,columns = column_names)
                  .assign(Time = time_points, CaL = "Candida", Nb = Nb, Agg =row["A"])
             )            

data_list=list()
for  key, row_s in df_data_sim.query("Nb == 0").groupby(["CaL","Nb"],group_keys=False):
    initial_CaL = key[0]*adjust_CaL*1/system_size*1e3
    data_list.append(row_s.reset_index().drop(columns=["CaL","Nb"]).assign(CaL = initial_CaL, Nb = 0))
for  key, row_s in df_data_candida_sim.query("Nb == 0").groupby(["Nb"],group_keys=False):
    data_list.append(row_s.reset_index().drop(columns=["Nb"]).assign(CaL = "Candida", Nb = 0))
df_scatter = pd.concat(data_list).sort_values(["CaL","Nb"])
df_effect = (pd.concat(df_liste)
             .assign(LDH = lambda df_:df_.LDH*1e3)
             .groupby(["CaL","Nb","Time"])["LDH"].agg(["mean","std"])
             .reset_index()
            )
fig = line_with_bands(data_frame=df_effect.assign(Nb= lambda df_:"<b>"+df_.Nb.map(str)+" µM</b>\t"), x="Time",y="mean",facet_col="CaL",color="Nb", error_y="std",width=1200,height=800, 
           labels={"mean":"LDH (ng/ml)","Time":"Time (h)","Nb":f"<b>{synthetic_nb}/{native_nb} concentration:</b>\t"},facet_col_spacing=0.1,facet_row_spacing=0.2,facet_col_wrap=4)
fig.update_xaxes(
    matches=None,
    showticklabels=True,
    range=[0,74],
    tickmode='array',
    tickvals=[0,24,48,72],  # Use all time points as tick values
)

fig.update_yaxes(
    matches=None,
    showticklabels=True,
    tickmode='array',
    tickvals=[0,500,1000,1500],  range=[0,1800]
)


scatter= px.scatter(df_scatter.assign(LDH = lambda df_:df_.Readout *1e3),x="Time",y="LDH",facet_col="CaL",facet_col_wrap=4)
scatter.for_each_trace(lambda trace:trace.update(name="DATA",marker=dict(size=10,color="black"),showlegend=False,legendgroup="DATA"))
for i in range(len(scatter.data)):
    fig.add_trace(scatter.data[i])
    
for yaxis in fig.layout:
    if "yaxis" in yaxis:
        fig.layout[yaxis].title.text = "<b>LDH (ng/ml)</b>"

for xaxis in fig.layout:
    if "xaxis" in xaxis:
        fig.layout[xaxis].title.text = "<b>Time (h)</b>"
fig.for_each_annotation(lambda anno: anno.update(text = " ".join(anno.text.__add__(" µM").split("=")[::-1])  if "Candida" != anno.text.split("=")[1] else "<i>C. albicans</i>-secreted CaL"))
fig.for_each_annotation(lambda anno: anno.update(text = "<b>"+anno.text+"</b>"))
fig.update_layout(legend=dict(y=.4,x=.6))
fig.show()
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"FIG_S3.svg"), format="svg")
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"FIG_S3.png"), format="png")

## Give parameter values by aggregate size in correct units

In [ ]:
time_dependent_variables = ['k_b', 'k_a', 'k_d', 'k_l', 'k_s']

parameter_table = (generate_df(RESULT_FOLDER)
 [time_dependent_variables+["A","alpha","beta"]]
.melt(id_vars="A", var_name="Parameter")
 .assign(value=lambda df_:df_.value.where(df_.Parameter.apply(lambda x:  x not in time_dependent_variables),df_.value/3600*time_scaling))
 .rename(columns={"value":"Parameter Value","A":"Aggregate Size"})
 .pivot(index='Parameter', columns='Aggregate Size')
)
display(parameter_table)

# Identifiability analysis using Profile Likelihood Methode

In [ ]:
AGG_SIZE = 8
all_parameters = generate_df(RESULT_FOLDER).query(f"A == {AGG_SIZE}").iloc[0].to_dict()

In [ ]:
CRITICAL_VALUE = chi2.ppf(0.95, 1)
time_dependent_variables = ['k_b', 'k_a', 'k_d', 'k_l', 'k_s']
AGG = "mean"
order_parameters = ['k_l', 'beta']
fitting_parameters = ['k_l', 'beta']
list_parameters = [all_parameters[i] for i in order_parameters]
no_fitting = {}
data = prepare_data_LDH(df_data_LDH)


def obj_wrapper_LDH(*parameters):
    SSE = objective_LDH(ldh_model.address, data, log_distance, *parameters)
    n_samples = len(data[0][0])
    return (n_samples/2)*np.log(SSE)


ldh_ci = (validation_interval(obj_wrapper_LDH, CRITICAL_VALUE,fitting_parameters, no_fitting, order_parameters, *list_parameters)
         )

order_parameters = ['k_a', 'k_c', 'k_l', 'k_d', 'alpha', 'beta']
fitting_parameters = ['k_a',  'k_d', 'alpha']
list_parameters = [all_parameters[i] for i in order_parameters]
no_fitting = {key:all_parameters[key] for key in ['k_l', 'beta','k_c']}
data = prepare_data_CaL(df_data_sim.query("Nb == 0.0"), all_parameters["beta"],agg=AGG)

def obj_wrapper_CaL(*parameters):
    SSE = objective_CAL(cal_model.address, data, distance, distance_max,*parameters)
    n_samples = np.sum([len(i[1]) for i in prepare_data_CaL(df_data_sim.query("Nb == 0.0"), all_parameters["beta"],agg=AGG)])
    return (n_samples/2)*np.log(SSE)


cal_ci = (validation_interval(obj_wrapper_CaL, CRITICAL_VALUE, fitting_parameters, no_fitting, order_parameters, *list_parameters)
         )


order_parameters = ['k_a', 'k_c', 'k_n','k_l','k_b', 'k_d','A', 'alpha', 'beta']
fitting_parameters = ['k_b']
list_parameters = [all_parameters[i] for i in order_parameters]
no_fitting = {key:all_parameters[key] for key in  ['k_a', 'k_c', 'k_l', 'k_d', 'k_n', 'A','alpha', 'beta']}
data = prepare_data_Nb(df_data_sim.query("Nb != 0.0"),df_data_pre.query("Nb != 0.0"),df_data_pre_co.query("Nb != 0.0"),df_data_post.query("Nb != 0.0"), all_parameters["beta"],agg=AGG)


def obj_wrapper_Nb(*parameters):
    SSE = objective_Nb(cal_nb_model.address, data, distance, distance_max,*parameters)
    n_samples = np.sum([len(i[-3]) for i in data["sim"]]+[len(i[-3]) for i in data["pre"]]+[len(i[-3]) for i in data["pre_co"]]+[len(i[-3]) for i in data["post"]])
    return (n_samples/2)*np.log(SSE)


nb_ci = (validation_interval(obj_wrapper_Nb, CRITICAL_VALUE, fitting_parameters, no_fitting,order_parameters, *list_parameters)
        )
all_parameters = generate_df(RESULT_FOLDER).query("A == 8").iloc[0].to_dict()
order_parameters = ['k_a', 'k_c', 'k_n','k_l','k_b', 'k_d','A', 'alpha', 'beta','k_s']
fitting_parameters = ['k_s']
list_parameters = [all_parameters[i] for i in order_parameters]
no_fitting = {key:all_parameters[key] for key in list(set(order_parameters)-set(fitting_parameters))}
data = prepare_data_Candida(df_data_candida_sim,df_data_candida_pre,df_data_candida_post,all_parameters["beta"])


def obj_wrapper_candida(*parameters):
    SSE = objective_Candida(candida_model.address, data, distance, distance_max,*parameters)
    n_samples = np.sum([len(i[-3]) for i in data["sim"]]+[len(i[-3]) for i in data["pre"]]+[len(i[-3]) for i in data["post"]])
    return (n_samples/2)*np.log(SSE)


candida_ci = (validation_interval(obj_wrapper_candida, CRITICAL_VALUE, fitting_parameters, no_fitting,order_parameters, *list_parameters)
              
             )
ci_s =(pd.concat([ldh_ci.query("Rejection == False").groupby("Parameter").apply(lambda grp_:pd.DataFrame((grp_["Test Value"].max(), grp_["Test Value"].min(), grp_["Mean"].iloc[0])).T.rename(columns = {0:"upper",1:"lower",2:"Mean"})),
                  cal_ci.query("Rejection == False").groupby("Parameter").apply(lambda grp_:pd.DataFrame((grp_["Test Value"].max(), grp_["Test Value"].min(), grp_["Mean"].iloc[0])).T.rename(columns = {0:"upper",1:"lower",2:"Mean"})),
                  nb_ci.query("Rejection == False").groupby("Parameter").apply(lambda grp_:pd.DataFrame((grp_["Test Value"].max(), grp_["Test Value"].min(), grp_["Mean"].iloc[0])).T.rename(columns = {0:"upper",1:"lower",2:"Mean"})),
                  candida_ci.query("Rejection == False").groupby("Parameter").apply(lambda grp_:pd.DataFrame((grp_["Test Value"].max(), grp_["Test Value"].min(), grp_["Mean"].iloc[0])).T.rename(columns = {0:"upper",1:"lower",2:"Mean"}))])
  .reset_index().drop(columns=["level_1"])
 .pipe(lambda df_: df_.assign(**{key:df_[key].where(df_.Parameter.apply(lambda x:  x not in time_dependent_variables),df_[key]/3600*time_scaling) for key in ["upper","lower","Mean"]}))
       .assign(Parameter= lambda df_:"$"+df_.Parameter.replace("alpha","alpha").replace("beta","beta")+"$")
 [["Parameter","Mean","lower","upper"]]
      .query("Parameter != 'k_c' and Parameter != 'k_n'")
      .set_index("Parameter")
     )
fig = px.scatter(ci_s.reset_index(), y="Parameter", x="Mean",log_x=True,
                 error_x=ci_s["upper"] - ci_s["Mean"],
                 error_x_minus=ci_s["Mean"] - ci_s["lower"],width=1000,height=400)
for yaxis in fig.layout:
    if "yaxis" in yaxis:
        fig.layout[yaxis].title.text = f"<b>Parameter</b>"

for xaxis in fig.layout:
    if "xaxis" in xaxis:
        fig.layout[xaxis].title.text = "<b>Mean Estimate (CI 95%)</b>"
fig.update_yaxes(showgrid=True,gridcolor="darkgray",gridwidth=1.8)
fig.update_xaxes(range=[-15,3],tickmode='array',gridcolor="darkgray",showgrid=True,gridwidth=1.8,
    tickvals=10**np.arange(-15,3,1).astype(float),)
fig.update_traces(marker=dict(size=12),error_x=dict(
    thickness=5,  
    width=5      
))


fig.show()
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_CI.svg"), format="svg")
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_CI.png"), format="png")
display(ci_s)

fig = px.scatter((pd.concat([ldh_ci,cal_ci,nb_ci,candida_ci])
                  .assign(TestValue = lambda df_:df_["Test Value"].where(df_.Parameter.apply(lambda x:  x not in time_dependent_variables),df_["Test Value"]/3600*time_scaling),
                          Parameter = lambda df_:"$"+df_.Parameter.map(str)+"$",
                          CI = lambda df_:100*chi2.cdf(df_["Test Statistic"], 1))),
                 x="TestValue",y="Test Statistic",color="Parameter",log_x=True,width=1000,height=400)
fig.update_xaxes(range=[-15,3],tickmode='array',tickvals=10**np.arange(-15,3,1).astype(float),)
fig.update_yaxes(range=[0,8])
fig.update_layout(legend=dict(orientation="v",x=1.0,xanchor="left",yanchor="top",y=1,title="<b>Parameter:</b>"))

for yaxis in fig.layout:
    if "yaxis" in yaxis:
        fig.layout[yaxis].title.text = f"<b>Test Statistic</b>"

for xaxis in fig.layout:
    if "xaxis" in xaxis:
        fig.layout[xaxis].title.text = "<b>Parameter Value</b>"

fig.add_hline(y=CRITICAL_VALUE,opacity=1.,visible=True,line=dict(width=3,dash="dash",color="rgba(255,0,0)"))
fig.add_annotation(
    x=-10,  # X-coordinate of the annotation
    y=CRITICAL_VALUE + 0.4,  # Adjust the Y-coordinate to position the text
    text="<b>95% Confidence</b>",  # The text you want to display
    showarrow=False,  # Hide the arrow
)
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_CI_INTERVALS.svg"), format="svg")
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_CI_INTERVALS.png"), format="png")
fig.show()

## Sensitivity analysis using Sobol variance based methode

In [ ]:
fitting_parameters = ['k_a', 'k_l','k_b', 'k_d', 'alpha', 'beta']
order_parameters = ['k_a', 'k_c', 'k_n','k_l','k_b', 'k_d','A', 'alpha', 'beta']
no_fitting_dict = {key:all_parameters[key] for key in ['A','k_n','k_c']}

boundaries = (pd.concat([ldh_ci.query("Rejection == False").groupby("Parameter").apply(lambda grp_:pd.DataFrame((grp_["Test Value"].max(), grp_["Test Value"].min(), grp_["Mean"].iloc[0])).T.rename(columns = {0:"upper",1:"lower",2:"Mean"})),
                  cal_ci.query("Rejection == False").groupby("Parameter").apply(lambda grp_:pd.DataFrame((grp_["Test Value"].max(), grp_["Test Value"].min(), grp_["Mean"].iloc[0])).T.rename(columns = {0:"upper",1:"lower",2:"Mean"})),
                  nb_ci.query("Rejection == False").groupby("Parameter").apply(lambda grp_:pd.DataFrame((grp_["Test Value"].max(), grp_["Test Value"].min(), grp_["Mean"].iloc[0])).T.rename(columns = {0:"upper",1:"lower",2:"Mean"})),
                  candida_ci.query("Rejection == False").groupby("Parameter").apply(lambda grp_:pd.DataFrame((grp_["Test Value"].max(), grp_["Test Value"].min(), grp_["Mean"].iloc[0])).T.rename(columns = {0:"upper",1:"lower",2:"Mean"}))])
              .reset_index().drop(columns=["level_1"])
              [["Parameter","Mean","lower","upper"]]
              .assign(Bound= lambda df_: df_.apply(lambda x: (x["lower"],x["upper"]),axis=1))
             )

ordered_boundaries = np.array([boundaries.query(f"Parameter == '{key}'").Bound.iloc[0] for key in fitting_parameters], dtype="float64")
problem = {
  'num_vars': len(fitting_parameters),
  'names': fitting_parameters,
  'bounds': ordered_boundaries
}

def f(x,times,cal,nb,col =-1):
    CaL_initial = cal/8*system_size*1e-3
    Nb_initial = nb*system_size*1e-3
    time_points = np.linspace(0,np.max(times),np.max(times)+1,dtype="float64")
    parameters = np.array(get_parameters(x, fitting_parameters, no_fitting_dict,order_parameters),dtype="float64")
    initials = np.array([CaL_initial,Nb_initial,0.0,1.0,0.0,0.0])
    time_points = np.linspace(0,np.max(times),np.max(times)+1)
    simulated_data = simulate(parameters, initials, time_points, cal_nb_model.address)[col,times]
    return simulated_data

SAMPLES = 2**12
df_list = list()
for CaL in [70]:
    for Nb in [0,16]:
        times = np.array([3,24,48])* time_scaling
        param_values = saltelli.sample(problem, SAMPLES)
        Y = np.vstack([f(params,times,CaL,Nb) for params in param_values])
        for i in range(len(times)):
            Si = sobol.analyze(problem, Y[:,i], print_to_console=False, parallel=False)
            df_list.append(pd.DataFrame(fitting_parameters,columns=["Parameter"]).assign(**{"<b>First Order</b>" : Si["S1"].flatten(), 
                                                                  "<b>Total Order</b>" : Si["ST"].flatten()},
                                                                 CaL = CaL,
                                                                 Nb = Nb,
                                                                 Time = times[i]))
            
df = pd.concat(df_list).melt(id_vars=["CaL","Nb","Time","Parameter"])
fig = px.bar(df.query(f"CaL == 70 and Parameter not in {['beta','k_l']}").assign(Parameter= lambda df_:"$"+df_.Parameter.replace({"beta":"β","alpha":"α"})+"$"),color="variable",y="Parameter",x="value", barmode='group',facet_col="Time",facet_row="Nb",                      width=1200,height=800, 
                      facet_col_wrap=2,facet_row_spacing=.15,facet_col_spacing=.12,
             labels={"variable":"<b>Sobol Sensitivity:</b>","value":"Index"})
fig.for_each_annotation(lambda anno: anno.update(text = anno.text.replace("Nb=","") + f" µM {synthetic_nb}; 70 µM CaL" if anno.text.split("=")[0] == "Nb" else anno.text.replace("Time=","Simultaneous addition; Time point: ") + " h"))
df.reset_index(drop=True).to_feather(os.path.join(PLOT_FIT_FOLDER,f"sobol_indicies.feather"))

fig.update_xaxes(
    range=[0,1.1],showticklabels=True,
)

for yaxis in fig.layout:
    if "yaxis" in yaxis:
        fig.layout[yaxis].title.text = "<b>Parameter</b>"

for xaxis in fig.layout:
    if "xaxis" in xaxis:
        fig.layout[xaxis].title.text = "<b>Sobol Index</b>"
fig.for_each_annotation(lambda anno: anno.update(text = "<b>"+anno.text+"</b>"))

fig.update_yaxes(showticklabels=True,range=[-1,4])
fig.show()

fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_SI_SYNTHETIC.svg"), format="svg")
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_SI_SYNTHETIC.png"), format="png")

In [ ]:
fitting_parameters = ['k_a', 'k_l','k_b', 'k_d', 'alpha', 'beta','k_s']
order_parameters = ['k_a', 'k_c', 'k_n','k_l','k_b', 'k_d','A', 'alpha', 'beta','k_s']
no_fitting_dict = {key:all_parameters[key] for key in ['A','k_n','k_c']}

boundaries = (pd.concat([ldh_ci.query("Rejection == False").groupby("Parameter").apply(lambda grp_:pd.DataFrame((grp_["Test Value"].max(), grp_["Test Value"].min(), grp_["Mean"].iloc[0])).T.rename(columns = {0:"upper",1:"lower",2:"Mean"})),
                  cal_ci.query("Rejection == False").groupby("Parameter").apply(lambda grp_:pd.DataFrame((grp_["Test Value"].max(), grp_["Test Value"].min(), grp_["Mean"].iloc[0])).T.rename(columns = {0:"upper",1:"lower",2:"Mean"})),
                  nb_ci.query("Rejection == False").groupby("Parameter").apply(lambda grp_:pd.DataFrame((grp_["Test Value"].max(), grp_["Test Value"].min(), grp_["Mean"].iloc[0])).T.rename(columns = {0:"upper",1:"lower",2:"Mean"})),
                  candida_ci.query("Rejection == False").groupby("Parameter").apply(lambda grp_:pd.DataFrame((grp_["Test Value"].max(), grp_["Test Value"].min(), grp_["Mean"].iloc[0])).T.rename(columns = {0:"upper",1:"lower",2:"Mean"}))])
              .reset_index().drop(columns=["level_1"])
              [["Parameter","Mean","lower","upper"]]
              .assign(Bound= lambda df_: df_.apply(lambda x: (x["lower"],x["upper"]),axis=1))
             )

ordered_boundaries = np.array([boundaries.query(f"Parameter == '{key}'").Bound.iloc[0] for key in fitting_parameters], dtype="float64")
problem = {
  'num_vars': len(fitting_parameters),
  'names': fitting_parameters,
  'bounds': ordered_boundaries
}

def f(x,times,nb,col =-4):
    candida_coverage = df_data_candida_sim.query("Time==72").Readout.mean()/all_parameters["beta"]
    Nb_initial = nb*system_size*1e-3
    time_points = np.linspace(0,np.max(times),np.max(times)+1,dtype="float64")
    parameters = np.array(get_parameters(x, fitting_parameters, no_fitting_dict,order_parameters),dtype="float64")
    initials = np.array([0.0,Nb_initial,0.0,candida_coverage,0.0,0.0,2e4,0.0,0.0])
    time_points = np.linspace(0,np.max(times),np.max(times)+1)
    simulated_data = simulate(parameters, initials, time_points, candida_model.address)[col,times]
    return simulated_data

SAMPLES = 2**12
df_list = list()
for Nb in [0,16]:
    times = np.array([3,24,48])* time_scaling
    param_values = saltelli.sample(problem, SAMPLES)
    Y = np.vstack([f(params,times,Nb) for params in param_values])
    for i in range(len(times)):
        Si = sobol.analyze(problem, Y[:,i], print_to_console=False, parallel=False)
        df_list.append(pd.DataFrame(fitting_parameters,columns=["Parameter"]).assign(**{"<b>First Order</b>" : Si["S1"].flatten(), 
                                                                  "<b>Total Order</b>" : Si["ST"].flatten()},
                                                             Nb = Nb,
                                                             Time = times[i]))

df = pd.concat(df_list).melt(id_vars=["Nb","Time","Parameter"])
fig = px.bar(df.query(f"Parameter not in {['beta','k_l']}").assign(Parameter= lambda df_:"$"+df_.Parameter.replace({"beta":"β","alpha":"α"})+"$"),color="variable",y="Parameter",x="value", barmode='group',facet_col="Time",facet_row="Nb",                      width=1200,height=800, 
                      facet_col_wrap=2,facet_row_spacing=.15,facet_col_spacing=.12,
             labels={"variable":"<b>Sobol Sensitivity:</b>","value":"Index"})
fig.for_each_annotation(lambda anno: anno.update(text = anno.text.replace("Nb=","") + f" µM {native_nb}" if anno.text.split("=")[0] == "Nb" else anno.text.replace("Time=","Simultaneous addition; Time point: ") + " h"))
df.reset_index(drop=True).to_feather(os.path.join(PLOT_FIT_FOLDER,f"sobol_indicies.feather"))

fig.update_xaxes(
    range=[0,1.1],showticklabels=True,
)

for yaxis in fig.layout:
    if "yaxis" in yaxis:
        fig.layout[yaxis].title.text = "<b>Parameter</b>"

for xaxis in fig.layout:
    if "xaxis" in xaxis:
        fig.layout[xaxis].title.text = "<b>Sobol Index</b>"
fig.for_each_annotation(lambda anno: anno.update(text = "<b>"+anno.text+"</b>"))

fig.update_yaxes(showticklabels=True,range=[-1,5])
fig.show()
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_SI_NATIVE.svg"), format="svg")
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_SI_NATIVE.png"), format="png")

In [ ]:
def simulate_data(funcptr,initials, time_points,**kwargs):
    '''Simulate model with a certain parmaater configuration'''
    order_parameters = ['k_a', 'k_c', 'k_n','k_l','k_b', 'k_d','A', 'alpha', 'beta']
    parameters = np.array([kwargs[op] for op in order_parameters])
    data = simulate(parameters, initials, time_points,funcptr)
    return data
CAL_CONC = 70
df_list = list()
column_names = ["CaL","Nb","CaLA","E_Alive","E_Dead","LDH", "NCAL","NCALA"]
for funcptr,df in [(cal_nb_with_binding_model.address,generate_df(RESULT_FOLDER))]:
    for row in df.to_dict(orient='records'):
        for delay in np.arange(0,24):
            row.update(delay=delay)
            candida_coverage= df_data_candida_sim.query("Time==72").Readout.mean()/df.beta.mean()
            time_delay = delay*time_scaling
            initials = np.array([CAL_CONC/row["A"]*1e-3*system_size,0,0.0,1.0,0.0,0.0,0.0,0.0])

            for Nb in [2,4,8,16]:
                if delay != 0.0:
                    time_points = np.linspace(0,time_delay,time_delay*10+1,dtype="float64")
                    post_initials = np.array([CAL_CONC/row["A"]*1e-3*system_size,0,0.0,1.0,0.0,0.0,0.0,0.0])
                    initials = simulate_data(funcptr,post_initials, time_points, **row)[:,-1]
                    df_list.append((pd.DataFrame(simulate_data(funcptr,post_initials, time_points, **row).T,columns = column_names)
                    .assign(Delay = delay, 
                      Time = time_points,  
                      initNb = Nb, 
                      Agg= row["A"])))
                setup_initials = np.copy(initials)
                time_max = 72*time_scaling
                time_points = np.linspace(0,time_max,time_max*10+1,dtype="float64")
                setup_initials[1] = Nb*system_size*1e-3
                df_list.append(pd.DataFrame(simulate_data(funcptr,setup_initials, time_points, **row).T,columns = column_names)
                          .assign(Delay = delay, 
                                  Time = time_points+delay,  
                                  initNb = Nb, 
                                  Agg= row["A"])
                 )
times = np.arange(0,24)
delay_array= [0,3,9,12]

data = pd.concat(df_list).assign(Nb = lambda df_:df_.Nb *1/(system_size*1e-3)).query("Time <= 48")
fig = px.line(data_frame=data.query(f"Delay == 0").assign(Agg = lambda df_:"<b>"+df_.Agg.map(int).map(str)+"</b>"), x="Time",y="Nb",color="Agg", 
                      width=1200,height=800, facet_col="initNb",
                      facet_col_wrap=2,facet_row_spacing=.2,facet_col_spacing=.12,
           labels={"Time":"Time (h)","initNb":f"<b>{synthetic_nb}</b>","Agg":"<b>Aggregate Size</b>"})
fig.for_each_annotation(lambda anno: anno.update(text = " ".join(anno.text.__add__(" µM").split("="))))

fig.update_xaxes(
    range=[0,24.3],showticklabels=True,
    tickmode='array',
    tickvals=np.arange(0,48,2)  # Use all time points as tick values
)

for yaxis in fig.layout:
    if "yaxis" in yaxis:
        fig.layout[yaxis].title.text = f"<b>{synthetic_nb} (µM)</b>"

for xaxis in fig.layout:
    if "xaxis" in xaxis:
        fig.layout[xaxis].title.text = "<b>Time (h)</b>"
fig.for_each_annotation(lambda anno: anno.update(text = "<b>"+anno.text+f"; CaL {CAL_CONC} µM</b>"))

fig.update_yaxes(showticklabels=True, matches=None,range=[0,16.2])
fig.show()
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_SYNTHETIC_NB_AGG.svg"), format="svg")
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_SYNTHETIC_NB_AGG.png"), format="png")

data = pd.concat(df_list).assign(NCALA = lambda df_:(df_.NCALA+df_.NCAL)/CAL_CONC*100 *1/(system_size*1e-3)*df_.Agg).query("Time <= 48")
fig = px.line(data_frame=data.query(f"Delay == 0").assign(Agg = lambda df_:"<b>"+df_.Agg.map(int).map(str)+"</b>"), x="Time",y="NCALA",color="Agg", 
                      width=1200,height=800, facet_col="initNb",
                      facet_col_wrap=2,facet_row_spacing=.2,facet_col_spacing=.12,
           labels={"Time":"Time (h)","initNb":f"<b>{synthetic_nb}</b>","Agg":"<b>Aggregate Size</b>"})
fig.for_each_annotation(lambda anno: anno.update(text = " ".join(anno.text.__add__(" µM").split("="))))

fig.update_xaxes(
    range=[0,24.3],showticklabels=True,
    tickmode='array',
    tickvals=np.arange(0,48,2)  # Use all time points as tick values
)

# Update y-axis titles for all subplots
for yaxis in fig.layout:
    if "yaxis" in yaxis:
        fig.layout[yaxis].title.text = f"<b>Neutralized CaL (%)</b>"

# Update x-axis titles for all subplots
for xaxis in fig.layout:
    if "xaxis" in xaxis:
        fig.layout[xaxis].title.text = "<b>Time (h)</b>"
fig.for_each_annotation(lambda anno: anno.update(text = "<b>"+anno.text+f"; CaL {CAL_CONC} µM</b>"))

fig.update_yaxes(showticklabels=True, matches=None,range=[0,100.1])
fig.show()
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_NCAL_AGG.svg"), format="svg")
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_NCAL_AGG.png"), format="png")

In [ ]:


data = (pd.concat(df_list).assign(Nb = lambda df_: df_.initNb-df_.Nb/(system_size*1e-3))
 .query("Time == 24 and Delay == 0")
 .assign(Binding = lambda df_:(df_.NCALA/(df_.NCALA+df_.NCAL))*100)
 [["Binding","Agg","initNb"]]
# .pivot(index='Agg', columns='initNb')
)

fig = px.line(data.assign(Agg=lambda df_:"<b>"+df_.Agg.map(int).map(str)+"</b>"), x="initNb",y="Binding",color="Agg", labels={"initNb":f"<b>Initial {synthetic_nb} (µM)</b>","Binding":"<b>Neutralized CaL in Aggregate (%)</b>","Agg":"<b>Aggregate Size</b>"})
fig.update_yaxes(range=[50,100])
fig.update_layout(legend=dict(orientation="v",x=1.02,xanchor="left",yanchor="top",y=1))
fig.show()
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_NCAL_%.svg"), format="svg")
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_NCAL_%.png"), format="png")

data = (pd.concat(df_list).assign(Nb = lambda df_: df_.initNb-df_.Nb/(system_size*1e-3))
 .query(f"Time in {[24]} and Delay == 0")
 .assign(Binding = lambda df_:(df_.NCALA/(df_.NCALA+df_.NCAL))*100)
 .assign(Binding= lambda df_:(1-df_.Binding/100)*df_.Agg+df_.Binding/100)
 [["Binding","Agg","initNb","Time"]]
# .pivot(index='Agg', columns='initNb')
)

fig = px.line(data.assign(Agg=lambda df_:"<b>"+df_.Agg.map(int).map(str)+"</b>"), x="initNb",y="Binding",color="Agg", labels={"initNb":f"<b>Initial {synthetic_nb} (µM)</b>","Binding":f"<b>Binding X Cal: 1 {synthetic_nb}</b>","Agg":"<b>Aggregate Size</b>"})
fig.update_yaxes(range=[0,8])
fig.update_layout(legend=dict(orientation="v",x=1.02,xanchor="left",yanchor="top",y=1))
fig.show()
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_NCAL_BINDING.svg"), format="svg")
fig.write_image(os.path.join(PLOT_FIT_FOLDER,f"EXTRA_NCAL_BINDING.png"), format="png")